In [5]:
%reload_ext autoreload
%autoreload 2
!pip install -r ../requirements.txt
!playwright install

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../requirements.txt'


In [4]:
from server.llm import (get_text_and_image_completions, parse_markdown_output, create_prompt_from_template)
from server.scraper import get_html_and_screenshot
import nest_asyncio
import asyncio
import time
import os


async def generate_variations(original_url, preview_url, project_dir, selector):

    os.makedirs(f"{project_dir}/generated", exist_ok=True)

    print('Getting the HTML and screenshot of the original page...')
    original_html, original_screenshot = await get_html_and_screenshot(original_url, selector)
    
    for iteration in range(0, 6):

        print(f'Running iteration {iteration}...')

        print('Running the assessment...')
        goals = [
            "Improve conversion rate",
            "Make the styles more consistent with the brand",
            "Improve the overall look and feel of the form"
        ]
        assessment_prompt = create_prompt_from_template(
            "server/generation_strategies/default/prompts/assessment.txt", 
            goals=goals,
            html=original_html.strip()
        )
        instructions = get_text_and_image_completions(assessment_prompt, image_data=original_screenshot)
        print(instructions)

        print('Generating CSS variation...')
        generation_prompt = create_prompt_from_template(
            "server/generation_strategies/default/prompts/generation.txt",
            instructions=instructions,
            html=original_html.strip()
        )
        raw_output = get_text_and_image_completions(generation_prompt, image_data=original_screenshot)
        parsed_output = parse_markdown_output(raw_output)
        with open(project_dir + f'generatedStyles.css', 'w') as text_file:
            text_file.write('\n'.join(parsed_output['css']))

        print("Sleeping for 3 second...")
        time.sleep(3)

        print('Getting the HTML and screenshot of the preview page...')
        generated_html, generated_screenshot = await get_html_and_screenshot(preview_url + '?generatedStyles', selector)

        print('Assessing the quality of the generated styles...')
        verification_prompt = create_prompt_from_template(
            "server/generation_strategies/default/prompts/verification.txt",
            html=generated_html.strip()
        )
        adjustments = get_text_and_image_completions(verification_prompt, image_data=generated_screenshot) 
        print(adjustments)

        print('Applying adjustments...')
        adjustment_prompt = create_prompt_from_template(
            "server/generation_strategies/default/prompts/adjustment.txt",
            adjustments=adjustments,
            html=generated_html.strip()
        )
        raw_output = get_text_and_image_completions(adjustment_prompt, image_data=generated_screenshot)
        parsed_output = parse_markdown_output(raw_output)

        with open(project_dir + f'generatedStyles.css', 'w') as text_file:
            text_file.write('\n'.join(parsed_output['css']))

        with open(project_dir + f'generated/styles{iteration}.css', 'w') as text_file:
            text_file.write('\n'.join(parsed_output['css']))

original_url = "https://main--wknd--hlxsites.hlx.live/"
preview_url = "http://localhost:3000/"
selector = '.featured-article.block'
project_dir = '/Users/tsaplin/Work/Sources/mystique-wknd/'

nest_asyncio.apply()
asyncio.get_event_loop().run_until_complete(generate_variations(original_url, preview_url, project_dir, selector))



Getting the HTML and screenshot of the original page...
Getting HTML and screenshot of https://main--wknd--hlxsites.hlx.live/...
Running iteration 0...
Running the assessment...
- Adjust the font size of the main heading (`h2`) to be larger for better readability.
- Use a consistent font family across all text elements for a cohesive look.
- Change the background color of the `div.text` to match the brand's primary color.
- Increase padding inside the `div.text` for better spacing.
- Modify the background color of the button (`a.button.primary`) to blend well with the rest of the design.
- Adjust the font weight of the article text (`div.text p`) for better readability.
- Change the line height of the article text to improve text legibility.
- Center align the button text for a more polished look.
- Ensure that the `img` element is fully responsive to maintain proportions on different screen sizes.
- Add border radius to the button for a more modern look.
Generating CSS variation...
Sl

Error: Page.goto: net::ERR_CONNECTION_REFUSED at http://localhost:3000/?generatedStyles
Call log:
navigating to "http://localhost:3000/?generatedStyles", waiting until "load"
